In this notebook we will use the nn class **Sequential** to model a CNN

We will use the single target network

# Import

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

In [2]:
import load_script

In [3]:
%load_ext autoreload
%autoreload 2

# Loading the dataset

In [4]:
train_dataset,target=load_script.load_dataset(train=True)

In [5]:
test_dataset,test_target=load_script.load_dataset(train=False)

In [6]:
target=target.long()
test_target=test_target.long()

# Added modules

In [7]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [8]:
class Tensor4D(nn.Module):
    def forward(self,input):
        return input.view(input.shape[0],1,*input.shape[1:])

In [9]:
class Tensor3D(nn.Module):
    def forward(self,input):
        return input.view(input.shape[0],input.shape[1],-1)

# Normalization

In [10]:
mean.shape

NameError: name 'mean' is not defined

In [ ]:
train_dataset.mean(0)

In [ ]:
mean=train_dataset.mean(0).view(1,28,-1)
std=train_dataset.std(0).view(1,28,-1)

In [ ]:
train_dataset=(train_dataset-mean)/std
test_dataset=(test_dataset-mean)/std

# Defining the network

### Single output

In [ ]:
class SingleOutput(nn.Module):
    def __init__(self):
        super(SingleOutput,self).__init__()

        self.conv1=nn.Conv1d(28,14,5)
        self.full1=nn.Linear(14*46,100)
        self.full2=nn.Linear(100,1)
        
    def forward(self,x):
        x=F.max_pool1d(F.relu(self.conv1(x)),1)
        x=x.view(-1)
        x=F.relu(self.full1(x.view(-1,14*46)))
        x=F.sigmoid(self.full2(x))
        
        return x

### Sequential

In [ ]:
# 50 -> 48 -> 24 -> 20 -> 10

In [ ]:
class Sequential(nn.Module):
    def __init__(self):
        super(Sequential,self).__init__()
        
        layers=[]
        layers.append(nn.Conv1d(28,28,3))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool1d(2))
        layers.append(nn.Conv1d(28,28,5))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool1d(2))
        layers.append(nn.Conv1d(28,28,5))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool1d(2))
        layers.append(Flatten())
        layers.append(nn.Linear(28*3,200))
        layers.append(nn.ReLU())
#         layers.append(nn.Linear(200,20))
#         layers.append(nn.ReLU())
        layers.append(nn.Linear(200,2))
#         layers.append(nn.Sigmoid())
        
        self.sequential=nn.Sequential(*layers)
        
    def forward(self,x):
        x=self.sequential(x)
        
        return x

### Convolution 2d

In [12]:
class CNN2D(nn.Module):
    def __init__(self):
        super(CNN2D,self).__init__()
        
        # 28*50
        layers=[]
        layers.append(nn.Conv1d(28,28,3))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool1d(2))
        
        # 28*24
        layers.append(nn.Conv1d(28,28,5))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool1d(2))
        
        # 28*10
        layers.append(Tensor4D())
        layers.append(nn.Conv2d(1,10,5))
        # 10*24*6
        layers.append(Tensor3D())
        # 10*144
        
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool1d(2))
        # 10*72
        
        layers.append(Flatten())
        layers.append(nn.Linear(10*72,200))
        layers.append(nn.ReLU())
        
        layers.append(nn.Linear(200,20))
        layers.append(nn.ReLU())
        
        layers.append(nn.Linear(20,2))        
        
        self.sequential=nn.Sequential(*layers)
        
    def forward(self,x):
        x=self.sequential(x)
        
        return x

# Training

In [29]:
def train_network(network,optimizer,verbose=False):
    
    criterion=nn.CrossEntropyLoss()
    mini_batch_size=79

    errors_test=[]
    for epoch in range(1001):

        total_loss=0
        output_target=torch.zeros(target.shape[0])
        for b in range(0,train_dataset.shape[0],mini_batch_size):


            train_element=train_dataset.narrow(0,b,mini_batch_size)
    #         train_element=train_element.view(1,28,-1)

            target_element=target.narrow(0,b,mini_batch_size)

            optimizer.zero_grad()


            out=net(train_element)
            output_target[b:b+mini_batch_size]=(out[:,1]>out[:,0]).data

            loss=criterion(out,target_element)
            loss.backward()
            optimizer.step()
            total_loss+=loss.data[0]

        output_test=torch.zeros(test_target.shape[0])
        out=net(test_dataset)
        output_test=(out[:,1]>out[:,0]).data.long()


    #     print(type(output_test))

        error_train=np.sum(list(output_target.long()==target.data))/target.shape[0]
        error_test=np.sum(list(output_test==test_target.data))/test_target.shape[0]
        errors_test.append(error_test)
        if epoch%10==0 and verbose:
            print('Epoch:',epoch,'Loss:',total_loss,'Correct:',str(error_train*100)[:5]+"%",
                 'Correct test:',str(error_test*100)[:5]+"%")
        if error_train>0.995:# or (epoch>80 and np.abs(np.mean(errors_test[-80:])-np.mean(errors_test[-25:]))<0.01):
            return epoch,errors_test[-1]

In [ ]:
class Train:
    
    def __init__(self,network,optimizer,verbose=False):
        self.network=network
        self.optimizer=optimizer
        self.verbose=verbose
        
    def __call__(self,repetitions=5):
        epochs=[]
        errors=[]
        for i in range(repetitions):
            for j in net.parameters():
                j.data.normal_(0, 0.01)
            print(i)
            epoch,error=train_network(self.network,self.optimizer,self.verbose)
            epochs.append(epoch)
            errors.append(error)
        
        return epochs,errors
    
    

In [ ]:
net=CNN2D()
optimizer = optim.Adam(net.parameters(), lr=0.0001,weight_decay=0)

train=Train(net,optimizer,True)
train()


0
Epoch: 0 Loss: 2.7725340723991394 Correct: 50.31% Correct test: 49.0%
Epoch: 10 Loss: 2.772398889064789 Correct: 50.31% Correct test: 49.0%
Epoch: 20 Loss: 2.7642818093299866 Correct: 51.26% Correct test: 49.0%
Epoch: 30 Loss: 2.6783082485198975 Correct: 63.92% Correct test: 49.0%
Epoch: 40 Loss: 2.554620862007141 Correct: 66.45% Correct test: 49.0%
Epoch: 50 Loss: 2.402999758720398 Correct: 70.25% Correct test: 57.0%
Epoch: 60 Loss: 2.233561098575592 Correct: 73.10% Correct test: 62.0%
Epoch: 70 Loss: 2.1059868037700653 Correct: 76.26% Correct test: 63.0%
Epoch: 80 Loss: 2.0083672404289246 Correct: 76.89% Correct test: 63.0%
Epoch: 90 Loss: 1.936369627714157 Correct: 78.79% Correct test: 65.0%
Epoch: 100 Loss: 1.8714019060134888 Correct: 78.48% Correct test: 66.0%
Epoch: 110 Loss: 1.811855435371399 Correct: 79.43% Correct test: 67.0%
Epoch: 120 Loss: 1.7556905150413513 Correct: 81.01% Correct test: 67.0%
Epoch: 130 Loss: 1.7061378955841064 Correct: 82.27% Correct test: 68.0%
Epoch: 

Epoch: 550 Loss: 0.24348251521587372 Correct: 99.36% Correct test: 78.0%
2
Epoch: 0 Loss: 2.772950530052185 Correct: 49.68% Correct test: 51.0%
Epoch: 10 Loss: 2.772908091545105 Correct: 49.68% Correct test: 51.0%
Epoch: 20 Loss: 2.772679924964905 Correct: 49.68% Correct test: 51.0%
Epoch: 30 Loss: 2.7715033888816833 Correct: 49.68% Correct test: 51.0%
Epoch: 40 Loss: 2.7661420106887817 Correct: 56.32% Correct test: 50.0%
Epoch: 50 Loss: 2.7512978315353394 Correct: 53.16% Correct test: 50.0%
Epoch: 60 Loss: 2.736502528190613 Correct: 54.43% Correct test: 52.0%
Epoch: 70 Loss: 2.7239893078804016 Correct: 56.64% Correct test: 51.0%
Epoch: 80 Loss: 2.7109817266464233 Correct: 60.44% Correct test: 49.0%
Epoch: 90 Loss: 2.6924540996551514 Correct: 62.34% Correct test: 51.0%
Epoch: 100 Loss: 2.6641107201576233 Correct: 64.55% Correct test: 49.0%
Epoch: 110 Loss: 2.6243008971214294 Correct: 64.55% Correct test: 49.0%
Epoch: 120 Loss: 2.5748459696769714 Correct: 67.08% Correct test: 50.0%
Epoc

In [15]:
net=CNN2D()

optimizer = optim.SGD(net.parameters(), lr=1)
optimizer = optim.Adam(net.parameters(), lr=0.0001,weight_decay=0)


criterion=nn.CrossEntropyLoss()
mini_batch_size=79

for epoch in range(1001):
    
    total_loss=0
    output_target=torch.zeros(target.shape[0])
    for b in range(0,train_dataset.shape[0],mini_batch_size):
        
        
        train_element=train_dataset.narrow(0,b,mini_batch_size)
#         train_element=train_element.view(1,28,-1)
        
        target_element=target.narrow(0,b,mini_batch_size)
        
        optimizer.zero_grad()
        

        out=net(train_element)
        output_target[b:b+mini_batch_size]=(out[:,1]>out[:,0]).data
        
        loss=criterion(out,target_element)
        loss.backward()
        optimizer.step()
        total_loss+=loss.data[0]
        
    output_test=torch.zeros(test_target.shape[0])
    out=net(test_dataset)
    output_test=(out[:,1]>out[:,0]).data.long()
        
    
#     print(type(output_test))
        
    error_train=np.sum(list(output_target.long()==target.data))/target.shape[0]
    error_test=np.sum(list(output_test==test_target.data))/test_target.shape[0]
    if epoch%10==0:
        print('Epoch:',epoch,'Loss:',total_loss,'Correct:',str(error_train*100)[:5]+"%",
             'Correct test:',str(error_test*100)[:5]+"%")

Epoch: 0 Loss: 2.826216459274292 Correct: 41.77% Correct test: 53.0%
Epoch: 10 Loss: 2.613489508628845 Correct: 65.50% Correct test: 52.0%
Epoch: 20 Loss: 2.4392541646957397 Correct: 69.93% Correct test: 56.0%
Epoch: 30 Loss: 2.2193009853363037 Correct: 74.05% Correct test: 55.0%
Epoch: 40 Loss: 1.9510942995548248 Correct: 78.79% Correct test: 57.0%
Epoch: 50 Loss: 1.6690470576286316 Correct: 83.54% Correct test: 61.0%
Epoch: 60 Loss: 1.3851625621318817 Correct: 87.65% Correct test: 58.0%
Epoch: 70 Loss: 1.115644097328186 Correct: 91.77% Correct test: 57.0%
Epoch: 80 Loss: 0.8598392009735107 Correct: 94.62% Correct test: 57.0%
Epoch: 90 Loss: 0.6713583469390869 Correct: 95.56% Correct test: 55.0%
Epoch: 100 Loss: 0.5844872444868088 Correct: 95.88% Correct test: 57.0%
Epoch: 110 Loss: 0.36556436121463776 Correct: 99.05% Correct test: 61.0%
Epoch: 120 Loss: 0.2529432997107506 Correct: 99.68% Correct test: 62.0%
Epoch: 130 Loss: 0.193000216037035 Correct: 100.0% Correct test: 62.0%
Epoch:

KeyboardInterrupt: 

In [ ]:
79*720

In [105]:
360*200

72000